In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg
from Mesh import Mesh

import matplotlib.pyplot as plt


In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
m = Mesh('data/coil_1.2_MM.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

print('Computing Laplacian')
K, M = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = linalg.eigh(K,M)

Computing Laplacian
Computing eigen values


In [4]:
# send all relevant numpy arrays to torch tensors
K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)
X = torch.from_numpy(m.verts).to(device)

In [5]:
# in the paper we used 50 eigenvalues so set k to 50
k = 50

In [6]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.SiLU())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# Initialize all layers (Xavier), final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
for p in model.net[-1].parameters():  # last Linear
    if p.ndim == 2:
        nn.init.normal_(p, std=1e-3)
    else:
        nn.init.zeros_(p)

# Helper: given A (N x k), return M-orthonormalized U = A (A^T M A)^{-1/2}
def m_orthonormalize(A, M):
    # A: (N, k), M: (N, N)
    # compute B = A^T M A (k x k)
    B = A.T @ (M @ A)  # k x k
    # symmetrize B
    B = 0.5*(B + B.T)
    # compute inverse sqrt of B via eigendecomposition (k small)
    s, Q = torch.linalg.eigh(B)  # s are eigenvalues
    # regularize small eigenvalues
    s_clamped = torch.clamp(s, min=1e-12)
    inv_sqrt = Q @ torch.diag(1.0/torch.sqrt(s_clamped)) @ Q.T
    U = A @ inv_sqrt
    return U

lr_start = 0.01
lr_end = 0.0001
max_epochs = 100_000
print_every = 1_000
loss_history = []

optimizer = optim.Adam(model.parameters(), lr=lr_start)
decay_factor = (lr_end / lr_start) ** (1 / max_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_factor)

for epoch in range(1, max_epochs+1):
    model.train()
    optimizer.zero_grad()
    A = model(X)  # N x k
    U = m_orthonormalize(A, M)  # U is M-orthonormal

    UKU = U.T @ (K @ U)
    UMU = U.T @ (M @ U)        # k x k
    
    orth_loss = torch.norm(UMU - torch.eye(k, device=device), p='fro')**2
    eig_loss = torch.norm(UKU, p='fro')**2

    loss = eig_loss + orth_loss
    loss.backward()
    optimizer.step()
    scheduler.step()
    loss_history.append(loss.item())
    if epoch % print_every == 0 or epoch == 1:
        print(f"Epoch {epoch:4d}, loss={loss.item():.6f}")

Epoch    1, loss=1368.665551
Epoch 1000, loss=2066.030754
Epoch 2000, loss=1912.940066
Epoch 3000, loss=1843.071664
Epoch 4000, loss=1794.844967
Epoch 5000, loss=1764.853150
Epoch 6000, loss=1743.827595
Epoch 7000, loss=1701.613736
Epoch 8000, loss=1676.294465
Epoch 9000, loss=1662.250554
Epoch 10000, loss=1646.344673
Epoch 11000, loss=1635.926793
Epoch 12000, loss=1624.898667
Epoch 13000, loss=1618.154888
Epoch 14000, loss=1611.124786
Epoch 15000, loss=1603.269654
Epoch 16000, loss=1598.331131
Epoch 17000, loss=1596.705161
Epoch 18000, loss=1590.091182
Epoch 19000, loss=1585.239140
Epoch 20000, loss=1584.945676
Epoch 21000, loss=1579.458842
Epoch 22000, loss=1575.208811
Epoch 23000, loss=1573.211510
Epoch 24000, loss=1568.768129
Epoch 25000, loss=1566.581510
Epoch 26000, loss=1566.542304
Epoch 27000, loss=1562.922383
Epoch 28000, loss=1560.906528
Epoch 29000, loss=1557.691642
Epoch 30000, loss=1556.045037
Epoch 31000, loss=1555.132180
Epoch 32000, loss=1552.917258
Epoch 33000, loss=15

In [8]:
np.set_printoptions(suppress=True, precision=6)

model.eval()
with torch.no_grad():
    A_final = model(X)

    U_final= m_orthonormalize(A_final, M)
    UKU = U_final.T @ (K @ U_final)

    print(f"The shape of U.t @ K @ U is: {UKU.shape}")

    approx_eigs = np.round(torch.diag(UKU).cpu().numpy(), 8)
    # For better comparison, we can compute Ritz values from subspace U by solving small generalized eigenproblem
    # (U^T K U) c = mu (U^T M U) c, but U^T M U = I so just eig of UK_final
    mu, Wsmall = np.linalg.eigh(UKU.cpu().numpy())
    mu = np.real(mu)
    # sort
    idx = np.argsort(mu)
    mu = mu[idx]
    print("\nLearned Ritz values (from U^T K U):", np.round(mu[:5], 6))
    print("Reference eigenvalues (first k):   ", np.round(eigvals[:5], 6))

The shape of U.t @ K @ U is: torch.Size([50, 50])

Learned Ritz values (from U^T K U): [0.001545 0.251101 0.279114 0.410295 0.528886]
Reference eigenvalues (first k):    [0.       0.007574 0.030308 0.068146 0.121208]


In [10]:
for i, j in zip(mu, eigvals):
    print(f"--- approximation: {np.round(i, 4)} actual: {np.round(j, 4)} ---")

--- approximation: 0.0015 actual: 0.0 ---
--- approximation: 0.2511 actual: 0.0076 ---
--- approximation: 0.2791 actual: 0.0303 ---
--- approximation: 0.4103 actual: 0.0681 ---
--- approximation: 0.5289 actual: 0.1212 ---
--- approximation: 0.5451 actual: 0.1892 ---
--- approximation: 0.731 actual: 0.2722 ---
--- approximation: 0.8444 actual: 0.3705 ---
--- approximation: 1.0492 actual: 0.4834 ---
--- approximation: 1.1186 actual: 0.6113 ---
--- approximation: 1.1889 actual: 0.754 ---
--- approximation: 1.3851 actual: 0.9117 ---
--- approximation: 1.7263 actual: 1.0836 ---
--- approximation: 1.7938 actual: 1.27 ---
--- approximation: 2.3232 actual: 1.4713 ---
--- approximation: 2.4727 actual: 1.687 ---
--- approximation: 2.5082 actual: 1.9172 ---
--- approximation: 2.6524 actual: 2.1605 ---
--- approximation: 2.8541 actual: 2.419 ---
--- approximation: 3.1354 actual: 2.6903 ---
--- approximation: 3.6757 actual: 2.9745 ---
--- approximation: 3.8079 actual: 3.2753 ---
--- approximation: 

In [ ]:
ax =  plt.figure(figsize=(10,10)).add_subplot(projection='3d')
surf = ax.plot_trisurf(m.verts[:,0], m.verts[:,1], m.verts[:,2], triangles = m.connectivity)
plt.show()